## Examples

In this examples document we will look at the plots and techniques we'll be using in the following exercises and activities.


#### Loading our dataset

In [10]:
# importing the necessary dependencies
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [11]:
# loading the Dataset with pandas
dataset = pd.read_csv('../../Datasets/computer_hardware.csv')

In [12]:
# looking at the dataset
dataset.head()

,vendor_name,model,myct,mmin,mmax,cach,chmin,chmax,prp,erp
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


Our example dataset contains:

- `vendor_name`: Name of the vendor
- `model`: model id
- `myct`: machine cycle time in nanoseconds
- `mmin`: minimum main memory in kilobytes
- `mmax`: maximum main memory in kilobytes
- `cach`: cache memory in kilobytes
- `chmin`: minimum channels in units
- `chmax`: maximum channels in units
- `prp`: published relative performance
- `erp`: estimated relative performance from the original article

---

### Simple Figure with a line

Basic plotting with lines, circles, etc. is a simple task when using Bokeh.   
By calling the `line` method of a given figure, we can add a line based on x and y coordinates in our visualization.

**Note**:   
If we simply want to plot our entries with the given index, we can create a new column and assign the dataset index in order to nicely access it.

In [13]:
# adding an index column to use it for the x-axis
dataset['index'] = dataset.index

In [14]:
# plotting the cache memory levels as line
plot = figure(title='Cache per Hardware', x_axis_label='Hardware index', y_axis_label='Cache Memory')
plot.line(dataset['index'], dataset['cach'], line_width=5)

show(plot)

### Scatter plot

Scatter plots can be used the same as line plots. It always depends on the scenario and data we are looking at which one should be used.   
Please refer back to the second chapter about different plots and data for this information.

In [17]:
# plotting the hardware cache as dots
plot = figure(title='Cache per Hardware', x_axis_label='Hardware', y_axis_label='Cache Memory')
plot.scatter(dataset['index'], dataset['cach'], size=5, color='red')

show(plot)

**Note**:   
We can provide a `color` argument with one of the pre-defined colors to quickly assign different colors to our plotted glyphs.

### Adding a legend

Legends display a mapping between e.g. lines in the plot and the according information like e.g. the hardware cache memory.

By adding a `legend_label` argument to the plot calls like `plot.line()`, we get a small box containing the information at the, by default, upper right corner.

In [15]:
# plotting cache memory and cycle time with legend
plot = figure(title='Attributes per Hardware', x_axis_label='Hardware index', y_axis_label='Attribute Value')
plot.line(dataset['index'], dataset['cach'], line_width=5, legend_label='Cache Memory')
plot.line(dataset['index'], dataset['myct'], line_width=5, color='red', legend_label='Cycle time in ns')

show(plot)

### Mutable Legend items 

When looking at the example from above, we can see that once we have several lines, the visualization can get cluttered.

We can give the user the ability to **"mute"**, meaning defocus, the clicked element in the legend. 

Adding a `muted_alpha` argument to the line plotting and adding a `click_policy` of `mute` to our legend element are the only two steps needed.

In [16]:
# adding mutability to the legend
plot = figure(title='Attributes per Hardware', x_axis_label='Hardware index', y_axis_label='Attribute Value')
plot.line(dataset['index'], dataset['cach'], line_width=5, legend_label='Cache Memory', muted_alpha=0.2)
plot.line(dataset['index'], dataset['myct'], line_width=5, color='red', legend_label='Cycle time in ns', muted_alpha=0.2)

plot.legend.click_policy="mute" 

show(plot)

### Color Mappers

Color mappers can map specific values to a given color in the selected spectrum.   
By providing the minimum and maximum value for a variable, we define the range in which colors are returned.

In [18]:
# adding color based on the mean price to our elements
from bokeh.models import LinearColorMapper

color_mapper = LinearColorMapper(palette='Magma256', low=min(dataset['cach']), high=max(dataset['cach'])) 

plot = figure(title='Cache per Hardware', x_axis_label='Hardware', y_axis_label='Cache Memory')
plot.scatter(dataset['index'], dataset['cach'], color={'field': 'y', 'transform': color_mapper}, size=10)

show(plot)

### DataSources

DataSources can be helpful in several cases like e.g. displaying a tooltip on hovering the data points.   
In most cases we can use pandas dataframes to feed data into our plot, but for certain features like tooltips, we have to use DataSource.

In [19]:
# using a ColumnDataSource to display a tooltip on hovering
from bokeh.models.sources import ColumnDataSource

data_source = ColumnDataSource(data=dict(
    vendor_name=dataset['vendor_name'],
    model=dataset['model'],
    cach=dataset['cach'],
    x=dataset['index'],
    y=dataset['cach']
))
    
TOOLTIPS=[
    ('Vendor', '@vendor_name'),
    ('Model', '@model'),
    ('Cache', '@cach')
]
    
plot = figure(title='Cache per Hardware', x_axis_label='Hardware'
              , y_axis_label='Cache Memory'
              , tooltips=TOOLTIPS)
plot.scatter('x', 'y', size=10, color='teal', source=data_source)

show(plot)

### Interactive Widget

In the example below, we first import the interact element from the ipywidgets library.   
This then allows us to define a new method and annotate it with the given @interact decorator.

The provided Value attribute tells the interact element which widget to used based on the data type of the argument. In our example we provide it a string which will give us a TextBox widget. 
We can refer to the table above to determine which Value data type will return which widget.

The print statement in the code above simply prints whatever has been entered in the textbox below the widget.

In [20]:
# importing the widgets
from ipywidgets import interact, interact_manual

# creating an input text
@interact(Value='Input Text')
def text_input(Value):
    print(Value)

interactive(children=(Text(value='Input Text', description='Value'), Output()), _dom_classes=('widget-interact…

**Note**   
The structure of the methods defined to use interact on always have the same structure. We will look at several examples in the following exercise.